# 잠재 일관성 모델

[잠재 일관성 모델(LCM)](https://hf.co/papers/2310.04378)은 픽셀 공간이 아닌 잠재 공간에서 역확산 과정을 직접 예측하여 빠르고 고품질의 이미지 생성을 가능하게 합니다. 즉, LCM은 노이즈가 있는 이미지에서 노이즈 없는 이미지를 예측하려고 시도하며, 이는 노이즈가 있는 이미지에서 반복적으로 노이즈를 제거하는 일반적인 확산 모델과 대조됩니다. 반복적인 샘플링 과정을 피함으로써 LCM은 20~30단계 대신 2~4단계 만에 고품질 이미지를 생성할 수 있습니다.

LCM은 사전 학습된 모델에서 증류되며, 이는 약 32시간의 A100 컴퓨팅이 필요합니다. 이를 가속화하기 위해 [LCM-LoRA](https://hf.co/papers/2311.05556)는 전체 모델에 비해 학습할 매개변수가 훨씬 적은 [LoRA 어댑터](https://huggingface.co/docs/peft/conceptual_guides/adapter#low-rank-adaptation-lora)를 학습합니다. LCM-LoRA는 학습된 후 확산 모델에 연결할 수 있습니다.

이 가이드에서는 작업에 대한 빠른 추론을 위해 LCM 및 LCM-LoRA를 사용하는 방법과 ControlNet 또는 T2I-Adapter와 같은 다른 어댑터와 함께 사용하는 방법을 보여줍니다.

> [!TIP]
> LCM 및 LCM-LoRA는 Stable Diffusion v1.5, Stable Diffusion XL 및 SSD-1B 모델에서 사용할 수 있습니다. 해당 체크포인트는 [잠재 일관성](https://hf.co/collections/latent-consistency/latent-consistency-models-weights-654ce61a95edd6dffccef6a8) 컬렉션에서 찾을 수 있습니다.

## 텍스트-이미지 변환

<hfoptions id="lcm-text2img">
<hfoption id="LCM">

LCM을 사용하려면 지원되는 모델에 대한 LCM 체크포인트를 [UNet2DConditionModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d-cond#diffusers.UNet2DConditionModel)에 로드하고 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체해야 합니다. 그런 다음 평소와 같이 파이프라인을 사용하고 텍스트 프롬프트를 전달하여 단 4단계 만에 이미지를 생성할 수 있습니다.

LCM을 사용할 때 염두에 두어야 할 몇 가지 참고 사항은 다음과 같습니다.

* 일반적으로 분류기 없는 안내를 위해 파이프라인 내부에서 배치 크기가 두 배로 늘어납니다. 그러나 LCM은 안내 임베딩을 사용하여 안내를 적용하며 배치 크기를 두 배로 늘릴 필요가 없으므로 추론 속도가 빨라집니다. 단점은 부정적인 프롬프트가 노이즈 제거 프로세스에 영향을 미치지 않기 때문에 LCM에서 작동하지 않는다는 것입니다.
* UNet이 학습된 값이기 때문에 `guidance_scale`의 이상적인 범위는 [3., 13.]입니다. 그러나 대부분의 경우 1.0 값으로 `guidance_scale`을 비활성화하는 것도 효과적입니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, LCMScheduler
import torch

unet = UNet2DConditionModel.from_pretrained(
    "latent-consistency/lcm-sdxl",
    torch_dtype=torch.float16,
    variant="fp16",
)
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", unet=unet, torch_dtype=torch.float16, variant="fp16",
).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

prompt = "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"
generator = torch.manual_seed(0)
image = pipe(
    prompt=prompt, num_inference_steps=4, generator=generator, guidance_scale=8.0
).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm/lcm_full_sdxl_t2i.png"/>
</div>

</hfoption>
<hfoption id="LCM-LoRA">

LCM-LoRA를 사용하려면 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체하고 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/loaders/lora#diffusers.loaders.StableDiffusionLoraLoaderMixin.load_lora_weights) 메서드로 LCM-LoRA 가중치를 로드해야 합니다. 그런 다음 평소와 같이 파이프라인을 사용하고 텍스트 프롬프트를 전달하여 단 4단계 만에 이미지를 생성할 수 있습니다.

LCM-LoRA를 사용할 때 염두에 두어야 할 몇 가지 참고 사항은 다음과 같습니다.

* 일반적으로 분류기 없는 안내를 위해 파이프라인 내부에서 배치 크기가 두 배로 늘어납니다. 그러나 LCM은 안내 임베딩을 사용하여 안내를 적용하며 배치 크기를 두 배로 늘릴 필요가 없으므로 추론 속도가 빨라집니다. 단점은 부정적인 프롬프트가 노이즈 제거 프로세스에 영향을 미치지 않기 때문에 LCM에서 작동하지 않는다는 것입니다.
* LCM-LoRA와 함께 안내를 사용할 수 있지만 `guidance_scale` 값이 높으면 매우 민감하며 생성된 이미지에 아티팩트가 발생할 수 있습니다. 가장 좋은 값은 [1.0, 2.0] 사이입니다.
* [stabilityai/stable-diffusion-xl-base-1.0](https://hf.co/stabilityai/stable-diffusion-xl-base-1.0)을 미세 조정된 모델로 교체합니다. 예를 들어 [animagine-xl](https://huggingface.co/Linaqruf/animagine-xl) 체크포인트를 사용하여 SDXL로 애니메이션 이미지를 생성해 보세요.

In [ ]:
import torch
from diffusers import DiffusionPipeline, LCMScheduler

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16",
    torch_dtype=torch.float16
).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl")

prompt = "Self-portrait oil painting, a beautiful cyborg with golden hair, 8k"
generator = torch.manual_seed(42)
image = pipe(
    prompt=prompt, num_inference_steps=4, generator=generator, guidance_scale=1.0
).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm/lcm_sdxl_t2i.png"/>
</div>

</hfoption>
</hfoptions>

## 이미지-이미지 변환

<hfoptions id="lcm-img2img">
<hfoption id="LCM">

이미지-이미지 변환에 LCM을 사용하려면 지원되는 모델에 대한 LCM 체크포인트를 [UNet2DConditionModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d-cond#diffusers.UNet2DConditionModel)에 로드하고 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체해야 합니다. 그런 다음 평소와 같이 파이프라인을 사용하고 텍스트 프롬프트와 초기 이미지를 전달하여 단 4단계 만에 이미지를 생성할 수 있습니다.

> [!TIP]
> 최상의 결과를 얻으려면 `num_inference_steps`, `strength` 및 `guidance_scale`에 대해 다른 값을 실험해 보세요.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image, UNet2DConditionModel, LCMScheduler
from diffusers.utils import load_image

unet = UNet2DConditionModel.from_pretrained(
    "SimianLuo/LCM_Dreamshaper_v7",
    subfolder="unet",
    torch_dtype=torch.float16,
)

pipe = AutoPipelineForImage2Image.from_pretrained(
    "Lykon/dreamshaper-7",
    unet=unet,
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png")
prompt = "Astronauts in a jungle, cold color palette, muted colors, detailed, 8k"
generator = torch.manual_seed(0)
image = pipe(
    prompt,
    image=init_image,
    num_inference_steps=4,
    guidance_scale=7.5,
    strength=0.5,
    generator=generator
).images[0]
image

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm-img2img.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">생성된 이미지</figcaption>
  </div>
</div>

</hfoption>
<hfoption id="LCM-LoRA">

이미지-이미지 변환에 LCM-LoRA를 사용하려면 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체하고 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/loaders/lora#diffusers.loaders.StableDiffusionLoraLoaderMixin.load_lora_weights) 메서드로 LCM-LoRA 가중치를 로드해야 합니다. 그런 다음 평소와 같이 파이프라인을 사용하고 텍스트 프롬프트와 초기 이미지를 전달하여 단 4단계 만에 이미지를 생성할 수 있습니다.

> [!TIP]
> 최상의 결과를 얻으려면 `num_inference_steps`, `strength` 및 `guidance_scale`에 대해 다른 값을 실험해 보세요.

In [ ]:
import torch
from diffusers import AutoPipelineForImage2Image, LCMScheduler
from diffusers.utils import make_image_grid, load_image

pipe = AutoPipelineForImage2Image.from_pretrained(
    "Lykon/dreamshaper-7",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")

init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png")
prompt = "Astronauts in a jungle, cold color palette, muted colors, detailed, 8k"

generator = torch.manual_seed(0)
image = pipe(
    prompt,
    image=init_image,
    num_inference_steps=4,
    guidance_scale=1,
    strength=0.6,
    generator=generator
).images[0]
image

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm-lora-img2img.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">생성된 이미지</figcaption>
  </div>
</div>

</hfoption>
</hfoptions>

## 인페인팅

인페인팅에 LCM-LoRA를 사용하려면 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체하고 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/loaders/lora#diffusers.loaders.StableDiffusionLoraLoaderMixin.load_lora_weights) 메서드로 LCM-LoRA 가중치를 로드해야 합니다. 그런 다음 평소와 같이 파이프라인을 사용하고 텍스트 프롬프트, 초기 이미지 및 마스크 이미지를 전달하여 단 4단계 만에 이미지를 생성할 수 있습니다.

In [ ]:
import torch
from diffusers import AutoPipelineForInpainting, LCMScheduler
from diffusers.utils import load_image, make_image_grid

pipe = AutoPipelineForInpainting.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")

init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png")
mask_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint_mask.png")

prompt = "concept art digital painting of an elven castle, inspired by lord of the rings, highly detailed, 8k"
generator = torch.manual_seed(0)
image = pipe(
    prompt=prompt,
    image=init_image,
    mask_image=mask_image,
    generator=generator,
    num_inference_steps=4,
    guidance_scale=4,
).images[0]
image

<div class="flex gap-4">
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/inpaint.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">초기 이미지</figcaption>
  </div>
  <div>
    <img class="rounded-xl" src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm-lora-inpaint.png"/>
    <figcaption class="mt-2 text-center text-sm text-gray-500">생성된 이미지</figcaption>
  </div>
</div>

## 어댑터

LCM은 LoRA, ControlNet, T2I-Adapter 및 AnimateDiff와 같은 어댑터와 호환됩니다. 이러한 어댑터에 LCM의 속도를 적용하여 특정 스타일의 이미지를 생성하거나 캐니 이미지와 같은 다른 입력으로 모델을 조건화할 수 있습니다.

### LoRA

[LoRA](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../using-diffusers/loading_adapters#lora) 어댑터는 몇 장의 이미지에서 새로운 스타일을 학습하고 사전 학습된 모델에 연결하여 해당 스타일의 이미지를 생성하도록 빠르게 미세 조정할 수 있습니다.

<hfoptions id="lcm-lora">
<hfoption id="LCM">

지원되는 모델에 대한 LCM 체크포인트를 [UNet2DConditionModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d-cond#diffusers.UNet2DConditionModel)에 로드하고 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체합니다. 그런 다음 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/loaders/lora#diffusers.loaders.StableDiffusionLoraLoaderMixin.load_lora_weights) 메서드를 사용하여 LoRA 가중치를 LCM에 로드하고 몇 단계 만에 스타일화된 이미지를 생성할 수 있습니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, LCMScheduler
import torch

unet = UNet2DConditionModel.from_pretrained(
    "latent-consistency/lcm-sdxl",
    torch_dtype=torch.float16,
    variant="fp16",
)
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", unet=unet, torch_dtype=torch.float16, variant="fp16",
).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights("TheLastBen/Papercut_SDXL", weight_name="papercut.safetensors", adapter_name="papercut")

prompt = "papercut, a cute fox"
generator = torch.manual_seed(0)
image = pipe(
    prompt=prompt, num_inference_steps=4, generator=generator, guidance_scale=8.0
).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm/lcm_full_sdx_lora_mix.png"/>
</div>

</hfoption>
<hfoption id="LCM-LoRA">

스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체합니다. 그런 다음 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/loaders/lora#diffusers.loaders.StableDiffusionLoraLoaderMixin.load_lora_weights) 메서드를 사용하여 LCM-LoRA 가중치와 사용하려는 스타일 LoRA를 로드합니다. `~loaders.UNet2DConditionLoadersMixin.set_adapters` 메서드를 사용하여 두 LoRA 어댑터를 결합하고 몇 단계 만에 스타일화된 이미지를 생성합니다.

In [ ]:
import torch
from diffusers import DiffusionPipeline, LCMScheduler

pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16",
    torch_dtype=torch.float16
).to("cuda")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl", adapter_name="lcm")
pipe.load_lora_weights("TheLastBen/Papercut_SDXL", weight_name="papercut.safetensors", adapter_name="papercut")

pipe.set_adapters(["lcm", "papercut"], adapter_weights=[1.0, 0.8])

prompt = "papercut, a cute fox"
generator = torch.manual_seed(0)
image = pipe(prompt, num_inference_steps=4, guidance_scale=1, generator=generator).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm/lcm_sdx_lora_mix.png"/>
</div>

</hfoption>
</hfoptions>

### ControlNet

[ControlNet](https://huggingface.co/docs/diffusers/main/en/using-diffusers/./controlnet)은 캐니 엣지, 자세 추정 또는 깊이와 같은 다양한 입력에 대해 학습할 수 있는 어댑터입니다. ControlNet을 파이프라인에 삽입하여 모델에 추가적인 조건화 및 제어를 제공하여 보다 정확한 생성을 할 수 있습니다.

[lllyasviel's](https://hf.co/lllyasviel) 리포지토리에서 다른 입력에 대해 학습된 추가 ControlNet 모델을 찾을 수 있습니다.

<hfoptions id="lcm-controlnet">
<hfoption id="LCM">

캐니 이미지에 대해 학습된 ControlNet 모델을 로드하고 [ControlNetModel](https://huggingface.co/docs/diffusers/main/en/api/models/controlnet#diffusers.ControlNetModel)에 전달합니다. 그런 다음 LCM 모델을 [StableDiffusionControlNetPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/controlnet#diffusers.StableDiffusionControlNetPipeline)에 로드하고 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체합니다. 이제 캐니 이미지를 파이프라인에 전달하고 이미지를 생성합니다.

> [!TIP]
> 최상의 결과를 얻으려면 `num_inference_steps`, `controlnet_conditioning_scale`, `cross_attention_kwargs` 및 `guidance_scale`에 대해 다른 값을 실험해 보세요.

In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image

from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, LCMScheduler
from diffusers.utils import load_image, make_image_grid

image = load_image(
    "https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png"
).resize((512, 512))

image = np.array(image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "SimianLuo/LCM_Dreamshaper_v7",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None,
).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

generator = torch.manual_seed(0)
image = pipe(
    "the mona lisa",
    image=canny_image,
    num_inference_steps=4,
    generator=generator,
).images[0]
make_image_grid([canny_image, image], rows=1, cols=2)

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm/lcm_full_sdv1-5_controlnet.png"/>
</div>

</hfoption>
<hfoption id="LCM-LoRA">

캐니 이미지에 대해 학습된 ControlNet 모델을 로드하고 [ControlNetModel](https://huggingface.co/docs/diffusers/main/en/api/models/controlnet#diffusers.ControlNetModel)에 전달합니다. 그런 다음 Stable Diffusion v1.5 모델을 [StableDiffusionControlNetPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/controlnet#diffusers.StableDiffusionControlNetPipeline)에 로드하고 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체합니다. [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/loaders/lora#diffusers.loaders.StableDiffusionLoraLoaderMixin.load_lora_weights) 메서드를 사용하여 LCM-LoRA 가중치를 로드하고 캐니 이미지를 파이프라인에 전달하고 이미지를 생성합니다.

> [!TIP]
> 최상의 결과를 얻으려면 `num_inference_steps`, `controlnet_conditioning_scale`, `cross_attention_kwargs` 및 `guidance_scale`에 대해 다른 값을 실험해 보세요.

In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image

from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, LCMScheduler
from diffusers.utils import load_image

image = load_image(
    "https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png"
).resize((512, 512))

image = np.array(image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image)

controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    safety_checker=None,
    variant="fp16"
).to("cuda")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")

generator = torch.manual_seed(0)
image = pipe(
    "the mona lisa",
    image=canny_image,
    num_inference_steps=4,
    guidance_scale=1.5,
    controlnet_conditioning_scale=0.8,
    cross_attention_kwargs={"scale": 1},
    generator=generator,
).images[0]
image

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm/lcm_sdv1-5_controlnet.png"/>
</div>

</hfoption>
</hfoptions>

### T2I 어댑터

[T2I-Adapter](https://huggingface.co/docs/diffusers/main/en/using-diffusers/./t2i_adapter)는 ControlNet보다 훨씬 가벼운 어댑터로, 사전 학습된 모델을 조건화하기 위한 추가 입력을 제공합니다. ControlNet보다 빠르지만 결과가 약간 나쁠 수 있습니다.

[TencentArc's](https://hf.co/TencentARC) 리포지토리에서 다른 입력에 대해 학습된 추가 T2I-Adapter 체크포인트를 찾을 수 있습니다.

<hfoptions id="lcm-t2i">
<hfoption id="LCM">

캐니 이미지에 대해 학습된 T2IAdapter를 로드하고 [StableDiffusionXLAdapterPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/adapter#diffusers.StableDiffusionXLAdapterPipeline)에 전달합니다. 그런 다음 LCM 체크포인트를 [UNet2DConditionModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d-cond#diffusers.UNet2DConditionModel)에 로드하고 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체합니다. 이제 캐니 이미지를 파이프라인에 전달하고 이미지를 생성합니다.

In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image

from diffusers import StableDiffusionXLAdapterPipeline, UNet2DConditionModel, T2IAdapter, LCMScheduler
from diffusers.utils import load_image, make_image_grid

# 고주파 세부 정보를 피하기 위해 저해상도에서 캐니 맵 감지
image = load_image(
    "https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png"
).resize((384, 384))

image = np.array(image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image).resize((1024, 1216))

adapter = T2IAdapter.from_pretrained("TencentARC/t2i-adapter-canny-sdxl-1.0", torch_dtype=torch.float16, variant="fp16").to("cuda")

unet = UNet2DConditionModel.from_pretrained(
    "latent-consistency/lcm-sdxl",
    torch_dtype=torch.float16,
    variant="fp16",
)
pipe = StableDiffusionXLAdapterPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    unet=unet,
    adapter=adapter,
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

prompt = "the mona lisa, 4k picture, high quality"
negative_prompt = "extra digit, fewer digits, cropped, worst quality, low quality, glitch, deformed, mutated, ugly, disfigured"

generator = torch.manual_seed(0)
image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=canny_image,
    num_inference_steps=4,
    guidance_scale=5,
    adapter_conditioning_scale=0.8,
    adapter_conditioning_factor=1,
    generator=generator,
).images[0]

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm-t2i.png"/>
</div>

</hfoption>
<hfoption id="LCM-LoRA">

캐니 이미지에 대해 학습된 T2IAdapter를 로드하고 [StableDiffusionXLAdapterPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/adapter#diffusers.StableDiffusionXLAdapterPipeline)에 전달합니다. 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체하고 [load_lora_weights()](https://huggingface.co/docs/diffusers/main/en/api/loaders/lora#diffusers.loaders.StableDiffusionLoraLoaderMixin.load_lora_weights) 메서드를 사용하여 LCM-LoRA 가중치를 로드합니다. 캐니 이미지를 파이프라인에 전달하고 이미지를 생성합니다.

In [ ]:
import torch
import cv2
import numpy as np
from PIL import Image

from diffusers import StableDiffusionXLAdapterPipeline, UNet2DConditionModel, T2IAdapter, LCMScheduler
from diffusers.utils import load_image, make_image_grid

# 고주파 세부 정보를 피하기 위해 저해상도에서 캐니 맵 감지
image = load_image(
    "https://hf.co/datasets/huggingface/documentation-images/resolve/main/diffusers/input_image_vermeer.png"
).resize((384, 384))

image = np.array(image)

low_threshold = 100
high_threshold = 200

image = cv2.Canny(image, low_threshold, high_threshold)
image = image[:, :, None]
image = np.concatenate([image, image, image], axis=2)
canny_image = Image.fromarray(image).resize((1024, 1024))

adapter = T2IAdapter.from_pretrained("TencentARC/t2i-adapter-canny-sdxl-1.0", torch_dtype=torch.float16, variant="fp16").to("cuda")

pipe = StableDiffusionXLAdapterPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    adapter=adapter,
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl")

prompt = "the mona lisa, 4k picture, high quality"
negative_prompt = "extra digit, fewer digits, cropped, worst quality, low quality, glitch, deformed, mutated, ugly, disfigured"

generator = torch.manual_seed(0)
image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=canny_image,
    num_inference_steps=4,
    guidance_scale=1.5,
    adapter_conditioning_scale=0.8,
    adapter_conditioning_factor=1,
    generator=generator,
).images[0]

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm-lora-t2i.png"/>
</div>

</hfoption>
</hfoptions>

### AnimateDiff

[AnimateDiff](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../api/pipelines/animatediff)는 이미지에 모션을 추가하는 어댑터입니다. 대부분의 Stable Diffusion 모델과 함께 사용할 수 있으며, 효과적으로 "비디오 생성" 모델로 전환합니다. 비디오 모델로 좋은 결과를 생성하려면 일반적으로 여러 프레임(16~24)을 생성해야 하며, 이는 일반적인 Stable Diffusion 모델에서는 매우 느릴 수 있습니다. LCM-LoRA는 각 프레임에 대해 4~8단계만 수행하여 이 프로세스를 가속화할 수 있습니다.

[AnimateDiffPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/animatediff#diffusers.AnimateDiffPipeline)을 로드하고 `MotionAdapter`를 전달합니다. 그런 다음 스케줄러를 [LCMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/lcm#diffusers.LCMScheduler)로 교체하고 `~loaders.UNet2DConditionLoadersMixin.set_adapters` 메서드를 사용하여 두 LoRA 어댑터를 결합합니다. 이제 파이프라인에 프롬프트를 전달하고 애니메이션 이미지를 생성할 수 있습니다.

In [ ]:
import torch
from diffusers import MotionAdapter, AnimateDiffPipeline, DDIMScheduler, LCMScheduler
from diffusers.utils import export_to_gif

adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5")
pipe = AnimateDiffPipeline.from_pretrained(
    "frankjoshua/toonyou_beta6",
    motion_adapter=adapter,
).to("cuda")

# 스케줄러 설정
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

# LCM-LoRA 로드
pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5", adapter_name="lcm")
pipe.load_lora_weights("guoyww/animatediff-motion-lora-zoom-in", weight_name="diffusion_pytorch_model.safetensors", adapter_name="motion-lora")

pipe.set_adapters(["lcm", "motion-lora"], adapter_weights=[0.55, 1.2])

prompt = "best quality, masterpiece, 1girl, looking at viewer, blurry background, upper body, contemporary, dress"
generator = torch.manual_seed(0)
frames = pipe(
    prompt=prompt,
    num_inference_steps=5,
    guidance_scale=1.25,
    cross_attention_kwargs={"scale": 1},
    num_frames=24,
    generator=generator
).frames[0]
export_to_gif(frames, "animation.gif")

<div class="flex justify-center">
    <img src="https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/lcm-lora-animatediff.gif"/>
</div>